# Iteration 4

In [2]:
# Must be included at the beginning of each new notebook. 
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('mdea117-Iteration-4').getOrCreate()

from pyspark.sql.functions import col, when, count
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import monotonically_increasing_id, row_number
from pyspark.sql.window import Window

#read in data
ghgdf = spark.read.load('Datasets/ghg emissions.csv', format='csv', header = 'true')
seadf = spark.read.load('Datasets/sea level changes.csv', format='csv', header = 'true')
tempdf = spark.read.load('Datasets/temp changes.csv', format='csv', header = 'true')

2.2

In [3]:
from pyspark.sql.functions import col, when, count, isnan
ghgdf.show()
#print(ghgdf.collect())
print(ghgdf.columns)
print((ghgdf.count(), len(ghgdf.columns)))
print("Expected:", ghgdf.count() * len(ghgdf.columns))
print("Nan:")
ghgdf.select([count(when(isnan(c), c)).alias(c) for c in ghgdf.columns]).show()
ghgdf.select([count(when(col(c).isNotNull() , c)).alias(c) for c in ghgdf.columns]).show()
print("Valid:", ghgdf.filter(col("CH4(MtCO2e)").isNotNull()).count() + ghgdf.filter(col("Years").isNotNull()).count() + ghgdf.filter(col("CO2(MtCO2e)").isNotNull()).count() + ghgdf.filter(col("N20(MtCO2e)").isNotNull()).count() + ghgdf.filter(col("Fgas(MtCO2e)").isNotNull()).count())

print(ghgdf.dtypes)

from pyspark.sql.types import DateType
ghgdf = spark.read.load('Datasets/ghg emissions.csv', format='csv', header = 'true')
pd = ghgdf.toPandas()
pd = pd.iloc[30:]
ghgdf = spark.createDataFrame(pd)
temp = ghgdf.withColumn('Years', from_unixtime(unix_timestamp('Years', 'dd/MM/yyy')).alias('Years').cast(DateType())) \
    .withColumn("CH4(MtCO2e)",col("CH4(MtCO2e)").cast("float")) \
    .withColumn("CO2(MtCO2e)",col("CO2(MtCO2e)").cast("float")) \
    .withColumn("N20(MtCO2e)",col("N20(MtCO2e)").cast("float")) \
    .withColumn("Fgas(MtCO2e)",col("Fgas(MtCO2e)").cast("float"))
ghgdf = temp       
print(ghgdf.dtypes)
ghgdf.show()

+----------+-----------+-----------+-----------+------------+
|     Years|CH4(MtCO2e)|CO2(MtCO2e)|N20(MtCO2e)|Fgas(MtCO2e)|
+----------+-----------+-----------+-----------+------------+
|01/12/1850|        600|        224|        104|        0.06|
|01/12/1851|        615|        231|     106.98|        0.06|
|01/12/1852|        630|        243|     109.96|        0.06|
|01/12/1853|      647.5|        256|     112.64|        0.06|
|01/12/1854|        665|        298|     115.33|        0.06|
|01/12/1855|        680|        308|     117.71|        0.06|
|01/12/1856|      697.5|        328|     120.09|        0.06|
|01/12/1857|        715|        332|     122.18|        0.06|
|01/12/1858|      732.5|        337|     123.97|        0.06|
|01/12/1859|      752.5|        358|     125.46|        0.06|
|01/12/1860|        770|        390|     126.65|        0.06|
|01/12/1861|      787.5|        406|     127.84|        0.06|
|01/12/1862|        805|        414|     128.74|        0.06|
|01/12/1

In [4]:
from pyspark.sql.functions import col, when, count
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import from_unixtime
from pyspark.sql.functions import monotonically_increasing_id, row_number
from pyspark.sql.window import Window

seadf = spark.read.load('Datasets/sea level changes.csv', format='csv', header = 'true')
seadf.show()
# print(seadf.collect())
print(seadf.columns)
print((seadf.count(), len(seadf.columns)))
print("Expected:", seadf.count() * len(seadf.columns))
print("Nan:")
seadf.select([count(when(isnan(c), c)).alias(c) for c in seadf.columns]).show()
print("Valid:", seadf.filter(col("GMSL(mm)").isNotNull()).count() + seadf.filter(col("Years").isNotNull()).count())
seadf.select([count(when(col(c).isNotNull() , c)).alias(c) for c in seadf.columns]).show()
print(seadf.dtypes)

from pyspark.sql.types import DateType, FloatType

seadf.show()
print(seadf.dtypes)
temp = seadf.withColumn('Years', from_unixtime(unix_timestamp('Years', 'dd/MM/yyy')).alias('Years').cast(DateType())) \
            .withColumn("GMSL(mm)",col("GMSL(mm)").cast(FloatType()))
temp.show()

seadf = temp
print(seadf.dtypes)

+----------+--------+
|     Years|GMSL(mm)|
+----------+--------+
|01/12/1880|  -158.7|
|01/12/1881|  -153.1|
|01/12/1882|  -169.9|
|01/12/1883|  -164.6|
|01/12/1884|  -143.7|
|01/12/1885|  -145.2|
|01/12/1886|  -147.6|
|01/12/1887|  -153.2|
|01/12/1888|  -151.1|
|01/12/1889|  -149.5|
|01/12/1890|  -147.5|
|01/12/1891|  -149.2|
|01/12/1892|    -146|
|01/12/1893|  -141.3|
|01/12/1894|    -151|
|01/12/1895|  -139.2|
|01/12/1896|  -146.8|
|01/12/1897|  -141.6|
|01/12/1898|  -132.2|
|01/12/1899|  -124.7|
+----------+--------+
only showing top 20 rows

['Years', 'GMSL(mm)']
(142, 2)
Expected: 284
Nan:
+-----+--------+
|Years|GMSL(mm)|
+-----+--------+
|    0|       0|
+-----+--------+

Valid: 282
+-----+--------+
|Years|GMSL(mm)|
+-----+--------+
|  142|     140|
+-----+--------+

[('Years', 'string'), ('GMSL(mm)', 'string')]
+----------+--------+
|     Years|GMSL(mm)|
+----------+--------+
|01/12/1880|  -158.7|
|01/12/1881|  -153.1|
|01/12/1882|  -169.9|
|01/12/1883|  -164.6|
|01/12/1884| 

In [5]:
from pyspark.sql.functions import col, when, count
tempdf.show()
print(tempdf.collect())
print(tempdf.columns)
print((tempdf.count(), len(tempdf.columns)))
print("Expected:", tempdf.count() * len(tempdf.columns))
print("Nan:")
tempdf.select([count(when(isnan(c), c)).alias(c) for c in tempdf.columns]).show()
print("Valid:", tempdf.filter(col("Land temp change ( C )").isNotNull()).count() + tempdf.filter(col("Years").isNotNull()).count()+ tempdf.filter(col("Ocean temp change ( C )").isNotNull()).count())
tempdf.select([count(when(col(c).isNotNull() , c)).alias(c) for c in tempdf.columns]).show()
from pyspark.sql.types import DateType, FloatType
temp = tempdf.withColumn('Years', from_unixtime(unix_timestamp('Years', 'dd/MM/yyy')).alias('Years').cast(DateType())) \
    .withColumn("Ocean temp change ( C )",col("Ocean temp change ( C )").cast(FloatType())) \
    .withColumn("Land temp change ( C )",col("Land temp change ( C )").cast(FloatType()))
print(temp.dtypes)
tempdf = temp

+----------+----------------------+-----------------------+
|     Years|Land temp change ( C )|Ocean temp change ( C )|
+----------+----------------------+-----------------------+
|01/12/1880|                 -0.34|                   0.01|
|01/12/1881|                 -0.63|                   0.05|
|01/12/1882|                 -0.01|                   0.09|
|01/12/1883|                 -1.29|                      0|
|01/12/1884|                 -0.61|                  -0.13|
|01/12/1885|                 -1.48|                  -0.17|
|01/12/1886|                 -0.55|                  -0.08|
|01/12/1887|                 -1.17|                  -0.35|
|01/12/1888|                 -0.84|                  -0.18|
|01/12/1889|                 -0.68|                   0.17|
|01/12/1890|                 -0.73|                  -0.22|
|01/12/1891|                 -0.87|                  -0.22|
|01/12/1892|                 -0.65|                   -0.1|
|01/12/1893|                    -2|     

Expected: 426
Nan:
+-----+----------------------+-----------------------+
|Years|Land temp change ( C )|Ocean temp change ( C )|
+-----+----------------------+-----------------------+
|    0|                     0|                      0|
+-----+----------------------+-----------------------+

Valid: 426
+-----+----------------------+-----------------------+
|Years|Land temp change ( C )|Ocean temp change ( C )|
+-----+----------------------+-----------------------+
|  142|                   142|                    142|
+-----+----------------------+-----------------------+

[('Years', 'date'), ('Land temp change ( C )', 'float'), ('Ocean temp change ( C )', 'float')]


2.4

In [6]:
ghgdf.printSchema()
ghgdf.describe().show()

print("Number of null values ghg:", (ghgdf.filter(col("Years").isNull()).count())+(ghgdf.filter(col("CH4(MtCO2e)").isNull()).count())+(ghgdf.filter(col("CO2(MtCO2e)").isNull()).count())+(ghgdf.filter(col("N20(MtCO2e)").isNull()).count())+(ghgdf.filter(col("Fgas(MtCO2e)").isNull()).count()))
print("Number of null values sea:", (seadf.filter(col("Years").isNull()).count()+ seadf.filter(col("GMSL(mm)").isNull()).count()))
print("Number of null values temp:", (tempdf.filter(col("Years").isNull()).count() + tempdf.filter(col("Land temp change ( C )").isNull()).count()+tempdf.filter(col("Ocean temp change ( C )").isNull()).count()))

seadf.printSchema()
seadf.describe().show()

tempdf.printSchema()
tempdf.describe().show()

root
 |-- Years: date (nullable = true)
 |-- CH4(MtCO2e): float (nullable = true)
 |-- CO2(MtCO2e): float (nullable = true)
 |-- N20(MtCO2e): float (nullable = true)
 |-- Fgas(MtCO2e): float (nullable = true)

+-------+------------------+------------------+------------------+------------------+
|summary|       CH4(MtCO2e)|       CO2(MtCO2e)|       N20(MtCO2e)|      Fgas(MtCO2e)|
+-------+------------------+------------------+------------------+------------------+
|  count|               142|               141|               142|               142|
|   mean| 4116.010563380281|12328.581560283688|1307.0876809778347|193.44260556767628|
| stddev|2299.0402480044495|10870.144142923922| 998.8046155483013|277.55631551470657|
|    min|            1152.5|            1080.0|            216.65|              0.06|
|    max|            8321.0|           36200.0|           3330.72|             999.0|
+-------+------------------+------------------+------------------+------------------+

Number of null 

3.1

In [7]:
print(((ghgdf.count(), len(ghgdf.columns))), ((seadf.count(), len(seadf.columns))), ((tempdf.count(), len(tempdf.columns))))

(142, 5) (142, 2) (142, 3)


3.2

In [8]:
from pyspark.sql.functions import mean as _mean
import pyspark.sql.functions as func
ghgdf.describe().show()
print("Number of null values ghg:", (ghgdf.filter(col("Years").isNull()).count())+(ghgdf.filter(col("CH4(MtCO2e)").isNull()).count())+(ghgdf.filter(col("CO2(MtCO2e)").isNull()).count())+(ghgdf.filter(col("N20(MtCO2e)").isNull()).count())+(ghgdf.filter(col("Fgas(MtCO2e)").isNull()).count()))
mean = ghgdf.select(func.round(_mean(col('CO2(MtCO2e)')),2).alias('mean')).collect()
mean = mean[0]['mean']
print(mean)
ghgdf = ghgdf.na.fill(mean)
print("Number of null values ghg:", (ghgdf.filter(col("Years").isNull()).count())+(ghgdf.filter(col("CH4(MtCO2e)").isNull()).count())+(ghgdf.filter(col("CO2(MtCO2e)").isNull()).count())+(ghgdf.filter(col("N20(MtCO2e)").isNull()).count())+(ghgdf.filter(col("Fgas(MtCO2e)").isNull()).count()))
ghgdf.describe().show()


seadf.describe().show()
print("Number of null values sea:", (seadf.filter(col("Years").isNull()).count()+ seadf.filter(col("GMSL(mm)").isNull()).count()))
mean = seadf.select(func.round(_mean(col('GMSL(mm)')),2).alias('mean')).collect()
mean = mean[0]['mean']
print(mean)
seadf = seadf.na.fill(mean)
print("Number of null values sea:", (seadf.filter(col("Years").isNull()).count()+ seadf.filter(col("GMSL(mm)").isNull()).count()))
seadf.describe().show()

outliers = seadf.filter(col('GMSL(mm)') > 1000).collect()
#print(outliers)
seadf = seadf.replace(outliers[0][1], mean)
outliers = seadf.filter(col('GMSL(mm)') > 1000).count()
print("outliers",outliers)

ghgdf = ghgdf.drop("Fgas(MtCO2e)")
ghgdf.show()

+-------+------------------+------------------+------------------+------------------+
|summary|       CH4(MtCO2e)|       CO2(MtCO2e)|       N20(MtCO2e)|      Fgas(MtCO2e)|
+-------+------------------+------------------+------------------+------------------+
|  count|               142|               141|               142|               142|
|   mean| 4116.010563380281|12328.581560283688|1307.0876809778347|193.44260556767628|
| stddev|2299.0402480044495|10870.144142923922| 998.8046155483013|277.55631551470657|
|    min|            1152.5|            1080.0|            216.65|              0.06|
|    max|            8321.0|           36200.0|           3330.72|             999.0|
+-------+------------------+------------------+------------------+------------------+

Number of null values ghg: 1
12328.58
Number of null values ghg: 0
+-------+------------------+------------------+------------------+------------------+
|summary|       CH4(MtCO2e)|       CO2(MtCO2e)|       N20(MtCO2e)|      

3.3

In [9]:
import pyspark.sql.functions as F
#new_tempdf = tempdf.withColumn("Average Temperature Change", func.col("Land temp change (C)")+ func.col("Ocean temp change ( C  )"))
newtempdf = tempdf.withColumn('Average Temperature Change', F.round(F.col('Land temp change ( C )') + F.col('Ocean temp change ( C )'),2))
newtempdf.show()

+----------+----------------------+-----------------------+--------------------------+
|     Years|Land temp change ( C )|Ocean temp change ( C )|Average Temperature Change|
+----------+----------------------+-----------------------+--------------------------+
|1880-12-01|                 -0.34|                   0.01|                     -0.33|
|1881-12-01|                 -0.63|                   0.05|                     -0.58|
|1882-12-01|                 -0.01|                   0.09|                      0.08|
|1883-12-01|                 -1.29|                    0.0|                     -1.29|
|1884-12-01|                 -0.61|                  -0.13|                     -0.74|
|1885-12-01|                 -1.48|                  -0.17|                     -1.65|
|1886-12-01|                 -0.55|                  -0.08|                     -0.63|
|1887-12-01|                 -1.17|                  -0.35|                     -1.52|
|1888-12-01|                 -0.84|        

3.4

In [10]:
finaldf = ghgdf.join(newtempdf, ["Years"]).join(seadf, ["Years"])
finaldf.show()
finaldf.describe().show()

+----------+-----------+-----------+-----------+----------------------+-----------------------+--------------------------+--------+
|     Years|CH4(MtCO2e)|CO2(MtCO2e)|N20(MtCO2e)|Land temp change ( C )|Ocean temp change ( C )|Average Temperature Change|GMSL(mm)|
+----------+-----------+-----------+-----------+----------------------+-----------------------+--------------------------+--------+
|1880-12-01|     1152.5|     1080.0|     216.65|                 -0.34|                   0.01|                     -0.33|  -158.7|
|1881-12-01|     1170.0|     1120.0|     219.63|                 -0.63|                   0.05|                     -0.58|  -153.1|
|1882-12-01|     1190.0|     1200.0|     222.31|                 -0.01|                   0.09|                      0.08|  -169.9|
|1883-12-01|     1210.0|     1280.0|     224.69|                 -1.29|                    0.0|                     -1.29|  -164.6|
|1884-12-01|     1227.5|     1310.0|     227.08|                 -0.61|     

3.5

In [11]:
print(finaldf.dtypes)
finaldf = finaldf.drop("Land temp change ( C )")
finaldf = finaldf.drop("Ocean temp change ( C )")
finaldf.show()

[('Years', 'date'), ('CH4(MtCO2e)', 'float'), ('CO2(MtCO2e)', 'float'), ('N20(MtCO2e)', 'float'), ('Land temp change ( C )', 'float'), ('Ocean temp change ( C )', 'float'), ('Average Temperature Change', 'float'), ('GMSL(mm)', 'float')]
+----------+-----------+-----------+-----------+--------------------------+--------+
|     Years|CH4(MtCO2e)|CO2(MtCO2e)|N20(MtCO2e)|Average Temperature Change|GMSL(mm)|
+----------+-----------+-----------+-----------+--------------------------+--------+
|1880-12-01|     1152.5|     1080.0|     216.65|                     -0.33|  -158.7|
|1881-12-01|     1170.0|     1120.0|     219.63|                     -0.58|  -153.1|
|1882-12-01|     1190.0|     1200.0|     222.31|                      0.08|  -169.9|
|1883-12-01|     1210.0|     1280.0|     224.69|                     -1.29|  -164.6|
|1884-12-01|     1227.5|     1310.0|     227.08|                     -0.74|  -143.7|
|1885-12-01|     1247.5|     1320.0|     229.16|                     -1.65|  -145.2

4.1

In [12]:
finaldf.describe().show()

#finaldf.stat.corr("CH4(MtCO2e)", "CO2(MtCO2e)") "method to do comparison between two single fields"

# from pyspark.ml.stat import Correlation
# from pyspark.ml.feature import VectorAssembler    "example code from stack overflow"
# # convert to vector column first
# vector_col = "corr_features"
# assembler = VectorAssembler(inputCols=finaldf.columns, outputCol=vector_col)
# df_vector = assembler.transform(finaldf).select(vector_col)
# # get correlation matrix
# matrix = Correlation.corr(df_vector, vector_col)

finaldf.describe().show()
finalpd = finaldf.toPandas()
print(finalpd.corr(method = "pearson"))



+-------+------------------+------------------+------------------+--------------------------+------------------+
|summary|       CH4(MtCO2e)|       CO2(MtCO2e)|       N20(MtCO2e)|Average Temperature Change|          GMSL(mm)|
+-------+------------------+------------------+------------------+--------------------------+------------------+
|  count|               142|               142|               142|                       142|               142|
|   mean| 4116.010563380281| 12328.58154984595|1307.0876809778347|       0.12887323920575666|-60.70542251308199|
| stddev|2299.0402480044495|10831.528943996065| 998.8046155483013|        0.9944078381148234| 66.61153971604983|
|    min|            1152.5|            1080.0|            216.65|                     -2.25|            -169.9|
|    max|            8321.0|           36200.0|           3330.72|                      2.88|              84.1|
+-------+------------------+------------------+------------------+--------------------------+---

4.2

In [13]:
from pyspark.sql import Row
from pyspark.sql.functions import pow, col
import pyspark.sql.functions as F

tempsqrt = finaldf.select("*", (pow(col("Average Temperature Change"), 0.5).alias("Average Temperature Change sqrt")))
tempsqrt = tempsqrt.drop("Average Temperature Change")
tempsqrt = tempsqrt.select("*", pow(col("CH4(MtCO2e)"), 0.5).alias("CH4(MtCO2e) sqrt"))
tempsqrt = tempsqrt.drop("CH4(MtCO2e)")
tempsqrt = tempsqrt.select("*", pow(col("CO2(MtCO2e)"), 0.5).alias("CO2(MtCO2e) sqrt"))
tempsqrt = tempsqrt.drop("CO2(MtCO2e)")
tempsqrt = tempsqrt.select("*", pow(col("N20(MtCO2e)"), 0.5).alias("N20(MtCO2e) sqrt"))
tempsqrt = tempsqrt.drop("N20(MtCO2e)")
tempsqrt = tempsqrt.select("*", pow(col("GMSL(mm)"), 0.5).alias("GMSL(mm) sqrt"))
tempsqrt = tempsqrt.drop("GMSL(mm)")
tempsqrt.show()
sq = tempsqrt.toPandas()
print(tempsqrt.select("CH4(MtCO2e) sqrt"))

tempcub = finaldf.select("*", (pow(col("Average Temperature Change"), 3).alias("Average Temperature Change cub")))
tempcub = tempcub.drop("Average Temperature Change")
tempcub = tempcub.select("*", pow(col("CH4(MtCO2e)"), 3).alias("CH4(MtCO2e) cub"))
tempcub = tempcub.drop("CH4(MtCO2e)")
tempcub = tempcub.select("*", pow(col("CO2(MtCO2e)"), 3).alias("CO2(MtCO2e) cub"))
tempcub = tempcub.drop("CO2(MtCO2e)")
tempcub = tempcub.select("*", pow(col("N20(MtCO2e)"), 3).alias("N20(MtCO2e) cub"))
tempcub = tempcub.drop("N20(MtCO2e)")
tempcub = tempcub.select("*", pow(col("GMSL(mm)"), 3).alias("GMSL(mm) cub"))
tempcub = tempcub.drop("GMSL(mm)")
tempcub.show()

cub = tempcub.toPandas()

#print(tempsqrt.select("CH4(MtCO2e) sqrt"))

import matplotlib.pyplot as plt
# finaldp = finaldf.toPandas()
# plt.hist(finaldp['CH4(MtCO2e)'])
# plt.title("CH4(MtCO2e)")
# plt.show()
# plt.hist(finaldp['CO2(MtCO2e)'])
# plt.title("CO2(MtCO2e)")
# plt.show()
# plt.hist(finaldp['N20(MtCO2e)'])
# plt.title("N20(MtCO2e)")
# plt.show()
# plt.hist(finaldp['Average Temperature Change'])
# plt.title("Average Temperature Change")
# plt.show()
# plt.hist(finaldp['GMSL(mm)'])
# plt.title("GMSL(mm)")
# plt.show()

#plt.hist(tempsqrt.select("CH4(MtCO2e) sqrt").collect())

# plt.hist(sq['CH4(MtCO2e) sqrt'])
# plt.title("CH4(MtCO2e) sqrt")
# plt.show()
# plt.hist(sq['CO2(MtCO2e) sqrt'])
# plt.title("CO2(MtCO2e) sqrt")
# plt.show()
# plt.hist(sq['N20(MtCO2e) sqrt'])
# plt.title("N20(MtCO2e) sqrt")
# plt.show()
# plt.hist(sq['Average Temperature Change sqrt'])
# plt.title("Average Temperature Change sqrt")
# plt.show()
# plt.hist(sq['GMSL(mm) sqrt'])
# plt.title("GMSL sqrt")
# plt.show()

plt.hist(cub['CH4(MtCO2e) cub'])
plt.title("CH4(MtCO2e) cub")
plt.show()
plt.hist(cub['CO2(MtCO2e) cub'])
plt.title("CO2(MtCO2e) cub")
plt.show()
plt.hist(cub['N20(MtCO2e) cub'])
plt.title("N20(MtCO2e) cub")
plt.show()
plt.hist(cub['Average Temperature Change cub'])
plt.title("Average Temperature Change cub")
plt.show()
plt.hist(cub['GMSL(mm) cub'])
plt.title("GMSL cub")
plt.show()

+----------+-------------------------------+------------------+------------------+------------------+-------------+
|     Years|Average Temperature Change sqrt|  CH4(MtCO2e) sqrt|  CO2(MtCO2e) sqrt|  N20(MtCO2e) sqrt|GMSL(mm) sqrt|
+----------+-------------------------------+------------------+------------------+------------------+-------------+
|1880-12-01|                            NaN| 33.94849039353591|32.863353450309965|14.719035087140881|          NaN|
|1881-12-01|                            NaN| 34.20526275297414| 33.46640106136302|14.819919192857041|          NaN|
|1882-12-01|            0.28284270931360533| 34.49637662132068| 34.64101615137755| 14.91006363361987|          NaN|
|1883-12-01|                            NaN|34.785054261852174| 35.77708763999664|14.989663186389688|          NaN|
|1884-12-01|                            NaN|35.035696082709705|36.193922141707716|15.069173893450651|          NaN|
|1885-12-01|                            NaN| 35.31996602489872|  36.3318

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>

6.3

In [24]:
print(finaldf.dtypes)

[('Years', 'date'), ('CH4(MtCO2e)', 'float'), ('CO2(MtCO2e)', 'float'), ('N20(MtCO2e)', 'float'), ('Average Temperature Change', 'float'), ('GMSL(mm)', 'float')]


Trial 1

In [26]:
import statsmodels.tsa.api as sm
import numpy as np
import pandas as pd

p = finaldf.toPandas()

schema = StructType([StructField('CH4(MtCO2e)', FloatType(), True),
                     StructField('CO2(MtCO2e)', FloatType(), True),
                     StructField('N20(MtCO2e)', FloatType(), True),
                     StructField('Average Temperature Change', FloatType(), True),
                     StructField('GMSL(mm)', FloatType(), True)])
                     
                    

@pandas_udf(schema, PandasUDFType.GROUPED_MAP)

##using the holt winters time series algorithm 
def holt_winters_time_series_udf(data):
  
    data.set_index('Years',inplace = True)
    ch4 = p['CH4(MtCO2e)']
    co2 = p['CO2(MtCO2e)']
    n20 = p['N20(MtCO2e)']
    tem = p['Average Temperature Change']
    gmsl = p['GMSL(mm)']
    

    ##the model
    mch4 = sm.ExponentialSmoothing(np.asarray(ch4),trend='add').fit()
    mco2 = sm.ExponentialSmoothing(np.asarray(co2),trend='add').fit()
    mn20 = sm.ExponentialSmoothing(np.asarray(n20),trend='add').fit()
    mtem = sm.ExponentialSmoothing(np.asarray(tem),trend='add').fit()
    mgmsl = sm.ExponentialSmoothing(np.asarray(gmsl),trend='add').fit()

    ##forecast values
    fch4 = pd.Series(mch4.forecast(50),name = 'ch4')
    fco2 = pd.Series(mco2.forecast(50),name = 'co2')
    fn20 = pd.Series(mn20.forecast(50),name = 'n20')
    ftem = pd.Series(mtem.forecast(50),name = 'tem')
    fgmsl = pd.Series(mgmsl.forecast(50),name = 'gmsl')
    
   
    return pd.DataFrame({'CH4(MtCO2e)': [fch4], 'CO2(MtCO2e)': [fco2],'N20(MtCO2e)': [fn20],'Average Temperature Change': [ftem],'GMSL(mm)': [fgmsl]})

fch4df = mch4\
    .apply(holt_winters_time_series_udf)
fco2df = mco2\
    .apply(holt_winters_time_series_udf)
fn20df = mn20\
    .apply(holt_winters_time_series_udf)
ftemdf = mtem\
    .apply(holt_winters_time_series_udf)


ImportError: No module named 'statsmodels'

Trial 2

In [22]:
def forecasts(p): 
    
    # instantiate the model, configure the parameters
    model = Prophet(
        interval_width=0.95,
        growth='linear',
        daily_seasonality=False,
        weekly_seasonality=False,
        yearly_seasonality=False,
        seasonality_mode='multiplicative'
    )
    
    # fit the model
    model.fit(p)
    
    future_pd = model.make_future_dataframe(
        periods=50, 
        freq='Y',
        include_history=True
    )
    
    results_pd = model.predict(future_pd)
    
    return results_pd

p = finaldf.toPandas()

r_schema = StructType([StructField("Years", DateType(), True),
                     StructField('CH4(MtCO2e)', FloatType(), True),
                     StructField('CO2(MtCO2e)', FloatType(), True),
                     StructField('N20(MtCO2e)', FloatType(), True),
                     StructField('Average Temperature Change', FloatType(), True),
                     StructField('GMSL(mm)', FloatType(), True)])
 
results = (
    finaldf.applyInPandas(forecasts, schema=r_schema)
    )

+----------+-----------+-----------+-----------+--------------------------+--------+
|     Years|CH4(MtCO2e)|CO2(MtCO2e)|N20(MtCO2e)|Average Temperature Change|GMSL(mm)|
+----------+-----------+-----------+-----------+--------------------------+--------+
|1880-12-01|     1152.5|     1080.0|     216.65|                     -0.33|  -158.7|
|1881-12-01|     1170.0|     1120.0|     219.63|                     -0.58|  -153.1|
|1882-12-01|     1190.0|     1200.0|     222.31|                      0.08|  -169.9|
|1883-12-01|     1210.0|     1280.0|     224.69|                     -1.29|  -164.6|
|1884-12-01|     1227.5|     1310.0|     227.08|                     -0.74|  -143.7|
|1885-12-01|     1247.5|     1320.0|     229.16|                     -1.65|  -145.2|
|1886-12-01|     1265.0|     1350.0|     230.95|                     -0.63|  -147.6|
|1887-12-01|     1282.5|     1420.0|     233.04|                     -1.52|  -153.2|
|1888-12-01|     1302.5|     1610.0|     234.82|                 

Linear regression trial 3

In [25]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['CH4(MtCO2e)','CO2(MtCO2e)','N20(MtCO2e)','Average Temperature Change','GMSL(mm)' ], outputCol = 'Features')
vfinaldf = vectorAssembler.transform(finaldf)
vfinaldf = vfinaldf.select(['Years', 'Features'])
vfinaldf.show()

from pyspark.sql.functions import percent_rank
from pyspark.sql import Window

df = vfinaldf.withColumn("rank", percent_rank().over(Window.partitionBy().orderBy("Years")))

train_data = df.where("rank <= .6").drop("rank")

test_data = df.where("rank > .6").drop("rank")


from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'Features', labelCol='Years')
lr_model = lr.fit(train_data)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

+----------+--------------------+
|     Years|            Features|
+----------+--------------------+
|1880-12-01|[1152.5,1080.0,21...|
|1881-12-01|[1170.0,1120.0,21...|
|1882-12-01|[1190.0,1200.0,22...|
+----------+--------------------+
only showing top 3 rows



IllegalArgumentException: 'requirement failed: Column Years must be of type NumericType but was actually of type DateType.'

From iteration 3 6.3- 8

In [26]:
from statsmodels.tsa.vector_ar.var_model import VAR
import pandas as pd

from pyspark.sql.functions import percent_rank
from pyspark.sql import Window

df = finaldf.withColumn("rank", percent_rank().over(Window.partitionBy().orderBy("Years")))

train_data = df.where("rank <= .6").drop("rank")

test_data = df.where("rank > .6").drop("rank")


p = train_data.toPandas()
q = test_data.toPandas()

df1 = p.drop(["Years"], axis = 1)
df1.index = pd.DatetimeIndex(p.Years).to_period('Y') #pyspark doesnt do indexing, so need to work in pandas

df2 = q.drop(["Years"], axis = 1)
df2.index = pd.DatetimeIndex(q.Years).to_period('Y')

final_model_train = VAR(endog=df1)
model_fit_train = final_model.fit()

final_model_test = VAR(endog=df2)
model_fit_test = final_model.fit()

forecast = model_fit_test.forecast(model_fit_test.y, steps= 50)

preddata1 = pd.DataFrame(forecast)

finalyr = df2.last_valid_index().strftime('%Y')
finalyr = int(finalyr)
preddata1.insert(0, 'Years', range(finalyr+1, finalyr + len(preddata1)+1))
preddata1['Years'] = pd.to_datetime(preddata1['Years'], format='%Y')
preddata = preddata1.drop(['Years'], axis=1)

preddata.index = pd.DatetimeIndex(preddata1.Years).to_period('Y')

preddata.columns = ['CH4(MtCO2e)','CO2(MtCO2e)', 'N20(MtCO2e)', "Average Temperatur Change (C)", 'GMSL(mm)']
#print(preddata)


dbwithpred = pd.concat([df, preddata])
#print(dbwithpred)

print(model_fit_test.summary())
print(model_fit_train.summary())
print(dbwithpred.corr(method = "pearson"))
print("mean:\n", dbwithpred.mean(), "\n min:\n", dbwithpred.min(), "\n max:\n", dbwithpred.max(), "\n std deviation:\n", dbwithpred.std(), "\n number of valid records:\n", dbwithpred.count(), "\n skew:\n", dbwithpred.skew())
Q1 = dbwithpred.quantile(0.25)
Q3 = dbwithpred.quantile(0.75)
IQR = Q3 - Q1
print("Outliers:\n", ((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).sum(), "Standard error of mean\n", dbwithpred.sem())

dbwithpred.plot(y='CH4(MtCO2e)', kind = 'line')
dbwithpred.plot(y='CO2(MtCO2e)', kind = 'line')
dbwithpred.plot(y='N20(MtCO2e)', kind = 'line')
dbwithpred.plot(y='Average Temperatur Change (C)', kind = 'line')
dbwithpred.plot(y='GMSL(mm)', kind = 'line')
dbwithpred.plot(kind = 'line')
plt.show()


#Patterns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# print("init data", db.corr(method = "pearson"))
# print("pred data", preddata.corr(method = "pearson"))
# print("final data",dbwithpred.corr(method = "pearson"))

ImportError: No module named 'statsmodels'

7.1

In [13]:
from pyspark.sql.functions import percent_rank
from pyspark.sql import Window

df = vfinaldf.withColumn("rank", percent_rank().over(Window.partitionBy().orderBy("Years")))

train_data = df.where("rank <= .6").drop("rank")

test_data = df.where("rank > .6").drop("rank")